In [7]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#### 1. First sight in data

In [2]:
train_data = pd.read_csv("datasets/titanic/train.csv")
test_data = pd.read_csv("datasets/titanic/test.csv")

In [4]:
print train_data.shape
print test_data.shape

(891, 12)
(418, 11)


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [9]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Useless**
1. Name
2. Ticket
3. Cabin
4. PassengerId

**Target**
Survived


In [90]:
target_column = 'Survived'
useless_column_list = ['Survived', 'Name', 'Ticket', 'Cabin',
                      'PassengerId']
useless_column_test_list = ['Name', 'Ticket', 'Cabin',
                      'PassengerId']
y_train = train_data['Survived']
X_train = train_data.drop(useless_column_list, axis=1)
X_test = test_data.drop(useless_column_test_list, axis=1)

In [91]:
X_test.shape

(418, 7)

#### Not Skewed data

In [20]:
y_train.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [25]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


#### Numeric or Scattered
Numeric: Age, SibSp, Parch and Fare

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


#### Missing Value：Age or Embarked
Age: Average
Embarked: Most Frequent

In [30]:
X_train['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [28]:
X_train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

#### Prepared Data

In [65]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from collections import Counter


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, selector):
        self._selector = selector
        return
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self._selector].values
    

class CategoricalImputer(TransformerMixin):
    def __init__(self):
        self.fill = {}
        return
    
    def fit(self, X, y=None):
        '''
        find the most frequent imputer in every column
        '''
        m, n = X.shape
        for col in xrange(n):
            col_data = X[:, col]
            cnt = Counter(col_data)
            self.fill[col] = cnt.most_common(1)[0][0]
            
        return self
    
    def transform(self, X, y=None):
        m, n = X.shape
        for col in xrange(n):
            for row in xrange(m):
                if X[row, col] == 'nan':
                    X[row, col] = self.fill[col]
        return X

In [92]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

numeric_attributes = ['Age', 'SibSp', 'Parch', 'Fare']
one_hot_attributes = ['Pclass']
scatter_attributes = ['Sex', ]
other_attributes = ['Embarked']

numeric_pipline = Pipeline([
    ('selector', DataFrameSelector(numeric_attributes)),
    ('imputer', Imputer(strategy='median')),
    ('scaler', StandardScaler()),
])
scatter_pipline = Pipeline([
    ('selector', DataFrameSelector(scatter_attributes)),
    ('imputer', CategoricalImputer()),
    ('label_binarizer', LabelBinarizer()),    
])
one_hot_pipeline = Pipeline([
    ('selector', DataFrameSelector(one_hot_attributes)),
    ('one_hot', OneHotEncoder())
])
other_pipeline = Pipeline([
    ('selector', DataFrameSelector(one_hot_attributes)),
    ('label_binarizer', LabelBinarizer())
])
full_pipeline = FeatureUnion(transformer_list = [
    ('numeric_pipeline', numeric_pipline),
    ('scatter_pipeline', scatter_pipline),
    ('one_hot_pipeline', one_hot_pipeline),
    ('other_pipeline', other_pipeline)
])

X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.fit_transform(X_test)

In [93]:
X_test_prepared.shape

(418, 11)

In [99]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
for_clf = RandomForestClassifier()
for_clf.fit(X_train_prepared, y_train)
result = for_clf.predict(X_test_prepared)
print result

passage_list = test_data['PassengerId']

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 0]


In [103]:
passage_list

0       892
1       893
2       894
3       895
4       896
5       897
6       898
7       899
8       900
9       901
10      902
11      903
12      904
13      905
14      906
15      907
16      908
17      909
18      910
19      911
20      912
21      913
22      914
23      915
24      916
25      917
26      918
27      919
28      920
29      921
       ... 
388    1280
389    1281
390    1282
391    1283
392    1284
393    1285
394    1286
395    1287
396    1288
397    1289
398    1290
399    1291
400    1292
401    1293
402    1294
403    1295
404    1296
405    1297
406    1298
407    1299
408    1300
409    1301
410    1302
411    1303
412    1304
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, dtype: int64

In [102]:
for i, key in passage_list.iteritems():
    print "{0},{1}".format(key, result[i])

892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1
901,0
902,0
903,0
904,1
905,0
906,1
907,1
908,0
909,0
910,0
911,0
912,1
913,0
914,1
915,1
916,1
917,0
918,1
919,0
920,0
921,0
922,0
923,0
924,0
925,0
926,0
927,0
928,1
929,0
930,0
931,1
932,0
933,0
934,0
935,1
936,1
937,0
938,0
939,0
940,1
941,0
942,1
943,0
944,1
945,1
946,0
947,0
948,0
949,0
950,0
951,1
952,0
953,0
954,0
955,1
956,1
957,1
958,1
959,0
960,0
961,1
962,1
963,0
964,0
965,0
966,1
967,0
968,0
969,1
970,0
971,1
972,1
973,0
974,0
975,0
976,0
977,0
978,0
979,1
980,0
981,1
982,0
983,0
984,1
985,0
986,0
987,0
988,1
989,0
990,1
991,0
992,1
993,0
994,0
995,0
996,0
997,0
998,0
999,0
1000,0
1001,0
1002,0
1003,0
1004,1
1005,1
1006,1
1007,0
1008,0
1009,1
1010,0
1011,1
1012,1
1013,0
1014,1
1015,0
1016,0
1017,0
1018,0
1019,1
1020,0
1021,0
1022,0
1023,0
1024,1
1025,0
1026,0
1027,0
1028,1
1029,0
1030,1
1031,0
1032,0
1033,1
1034,0
1035,0
1036,0
1037,0
1038,1
1039,0
1040,0
1041,0
1042,1
1043,0
1044,0
1045,1
1046,0
1047,0
1048,1
1049,1
10